In [15]:
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np

# Built In Imports
from datetime import datetime
from glob import glob
import warnings
import IPython

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from typing import List, Optional
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import patches
from pathlib import Path
from pycocotools.coco import COCO

import csv 

LABEL_COLORS = [px.colors.label_rgb(px.colors.convert_to_RGB_255(x)) for x in sns.color_palette("Spectral", 10)]
LABEL_COLORS_WOUT_NO_FINDING = LABEL_COLORS[:8]+LABEL_COLORS[9:]

coco = COCO('/opt/ml/dataset/test.json')

classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]


def plot_bboxes(
  image_file: str,
  bboxes: List[List[float]],
  scores: List[List[int]],
  xywh: bool = False,
  labels: Optional[List[str]] = None
  
) -> None:
    """
    Args:
      image_file: str specifying the image file path
      bboxes: list of bounding box annotations for all the detections
      xywh: bool, if True, the bounding box annotations are specified as
        [xmin, ymin, width, height]. If False the annotations are specified as
        [xmin, ymin, xmax, ymax]. If you are unsure what the mode is try both
        and check the saved image to see which setting gives the
        correct visualization.

    """
    color_dict = {0:'red', 1:'orange', 2:'yellow', 3:'green', 4:'blue', 5:'white', 6:'black', 7:'pink', 8:'navy', 9:'purple'}

    fig = plt.figure()

    # add axes to the image
    ax = fig.add_axes([0, 0, 1, 1])

    image_folder = Path(image_file).parent

    # read and plot the image
    image = plt.imread(image_file)
    plt.imshow(image)

    # Iterate over all the bounding boxes
    for i, bbox in enumerate(bboxes):
        if xywh:
          xmin, ymin, w, h = bbox
        else:
          xmin, ymin, xmax, ymax = bbox
          w = xmax - xmin
          h = ymax - ymin

        # add bounding boxes to the image
        box = patches.Rectangle(
            (xmin, ymin), w, h, edgecolor= color_dict[int(labels[i])], facecolor="none", linewidth = 2.0
        )

        ax.add_patch(box)

        s = classes[int(labels[i])] + ' ' + str(round(scores[i], 2))

        if labels is not None:
          rx, ry = box.get_xy()
          cx = rx + box.get_width()/2.0
          cy = ry + box.get_height()/8.0
          l = ax.annotate(
            s,
            (cx, cy),
            fontsize=8,
            fontweight="bold",
            color="white",
            ha='center',
            va='center'
          )
          l.set_bbox(
            dict(facecolor=color_dict[int(labels[i])], alpha=0.5, edgecolor=color_dict[int(labels[i])])
          )

def import_csv_file(file_name: str) -> None: 
    file = open(file_name, 'r') 
    data = list(csv.reader(file, delimiter=','))
    num_img = len(data)

    labels, scores, boxes = [], [], [] 
    for i in range(1, num_img): # for number of images
        label, score, box = [], [], []

        one_img = data[i][0].split(' ') 

        for j in range(0, len(one_img)-1, 6): 
            label.append(int(one_img[j]))
            score.append(float(one_img[j+1]))
            box.append(([float(one_img[k]) for k in range(j+2, j+6)])) 

        labels.append(label)
        scores.append(score)
        boxes.append(box)

    return labels, scores, boxes 

def show_bbox_on_train_image(image_id : int, file_name : str) -> None:

    l, s, b = import_csv_file(file_name)

    image_info = coco.loadImgs(image_id)[0]

    labels = l[image_id]
    bboxes = b[image_id]
    scores = s[image_id]

    plot_bboxes(image_file = '/opt/ml/dataset/test/'+f'{image_id:04d}.jpg', bboxes = bboxes, scores = scores, xywh= True, labels = labels)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
show_bbox_on_train_image(5, 'data_paths/swinT_cascade_12_gyubo.csv')

In [ ]:
show_bbox_on_train_image(5, 'data_paths/swinT_cascade_12_sangmo.csv')